In [1]:
import numpy as np
from scipy import spatial,stats
import pandas as pd
import time
import math
import matplotlib.pyplot as plt
%matplotlib inline

In [19]:
window_items = [1, 2, 3, 4, 5, 257, 7, 8, 9, 10, 11, 13, 205, 17, 211, 213, 215, 219, 220, 234, 236, 239, 250, 255]
target_items = [12, 207, 216, 235, 241] 

In [20]:
import os,sys
my_path = os.getcwd()
print (my_path)
path = os.path.join(my_path, "../data/ratings.txt")
print (path)

/home/surya/surya_114cs0121/AttackProfileDetectionRS/filmtrust/User-Based-Recommendation
/home/surya/surya_114cs0121/AttackProfileDetectionRS/filmtrust/User-Based-Recommendation/../data/ratings.txt


In [21]:
train = pd.read_csv(path,sep=' ',header=None,names=['userId','itemId','rating'])

In [22]:
train.describe()

,userId,itemId,rating
count,35497.000000,35497.000000,35497.000000
mean,744.247824,265.524946,3.002803
std,433.199457,346.854769,0.918692
min,1.000000,1.000000,0.500000
25%,356.000000,12.000000,2.500000
50%,735.000000,216.000000,3.000000
75%,1130.000000,251.000000,4.000000
max,1508.000000,2071.000000,4.000000


### Actual Average Rating of each item 

In [23]:
calculated_Rating = {}
items = train['itemId'].unique()

#rating
R = train.groupby(['itemId'])['rating'].mean()
for i in items:
#     print (i)
    calculated_Rating[i] = R[i]
    
# print (calculated_Rating)

In [24]:
for t in target_items:
    print (t,' : ',calculated_Rating[i])

12  :  3.0
207  :  3.0
216  :  3.0
235  :  3.0
241  :  3.0


#### with simple train-test(previously used split) 
test = pd.DataFrame()  
for t_i in target_items:  
    temp_df = train[train['itemId'] == t_i]  
    train =  train.drop(train[train['itemId'] == t_i].index)  
    test = pd.concat([temp_df, test]).drop_duplicates(keep=False)  

### Checking by only taking users who have rated more than 100 items :Conclusion : strong attack 

In [25]:
temp_test = pd.DataFrame()  
for t_i in target_items:  
    temp_df = train[train['itemId'] == t_i]  
    train =  train.drop(train[train['itemId'] == t_i].index)  
    temp_test = pd.concat([temp_df, temp_test]).drop_duplicates(keep=False)  
    
# print (temp_test.shape)

uu = []  
test = pd.DataFrame()  

for index1,i in enumerate(temp_test.itertuples(),0):  
    
    usr=int(i.userId)  
    mov=int(i.itemId)  
#     print (index1)    
    list_of_movies_current_user_has_rated=train[train['userId']==usr]  
    if(list_of_movies_current_user_has_rated.shape[0]<100):  
        pass                  #test = test.drop(test.index[index1])  
    else:  
        k = pd.Series([i.userId,i.itemId,i.rating])  
        test = test.append(k, ignore_index=True)  
        uu.append(usr)  
print (len(uu))  

print (test.shape)  
test.columns = ['userId','itemId','rating']  
print (test.head(5))  

37
(37, 3)
   userId  itemId  rating
0   161.0   241.0     2.5
1   670.0   241.0     3.5
2   845.0   241.0     3.5
3  1187.0   241.0     2.5
4  1199.0   241.0     3.5


In [26]:
test.shape

(37, 3)

### testing code of divising test/target data
table = [['Heading1', 'Heading2'], [1 , 1],[1 , 2],[1 , 3],[1 , 4], [1,5],[2 , 1],[2 , 2],[2 , 3],[2 , 4],[3, 1],[3, 2],[3, 3],[3, 4]]  
headers = table.pop(0)  
df = pd.DataFrame(table, columns=headers)

In [27]:
dfs = {n: g for n, g in test.groupby(test.groupby('itemId').cumcount() % 4)} 
test = pd.concat([dfs[0],dfs[1]], ignore_index=False)
train = pd.concat([train,dfs[2],dfs[3]], ignore_index=False)
# del_data = pd.concat([dfs[6],dfs[7],dfs[8],dfs[9]], ignore_index=False)

In [28]:
dfs[1]

,userId,itemId,rating
1,670.0,241.0,3.5
5,1333.0,241.0,1.0
7,199.0,235.0,1.5
12,199.0,216.0,0.5
16,1199.0,216.0,3.0
18,188.0,207.0,2.5
22,670.0,207.0,3.0
26,1187.0,207.0,0.5
30,272.0,12.0,2.0
34,1199.0,12.0,4.0


In [29]:
train.shape[0]

32143

In [33]:
test.shape[0] + train.shape[0] 

32164

In [31]:
test.tail(5)

,userId,itemId,rating
18,188.0,207.0,2.5
22,670.0,207.0,3.0
26,1187.0,207.0,0.5
30,272.0,12.0,2.0
34,1199.0,12.0,4.0


In [32]:
train.tail(5)

,userId,itemId,rating
20,272.0,207.0,0.5
24,845.0,207.0,4.0
28,1249.0,207.0,3.0
32,1039.0,12.0,3.0
36,1333.0,12.0,0.5


In [169]:
del_data.tail(5)

,userId,itemId,rating
35,1249.0,12.0,2.5
24,845.0,207.0,4.0
36,1333.0,12.0,0.5
25,1039.0,207.0,3.5
26,1187.0,207.0,0.5


### Finding error for target items in actual data

In [170]:
target_item_error_in_actual_data = dict((t_i,[0,0]) for t_i in target_items)
print (target_item_error_in_actual_data)

{216: [0, 0], 241: [0, 0], 235: [0, 0], 12: [0, 0], 207: [0, 0]}


In [171]:
t1=time.time()
for index,i in enumerate(test.itertuples(),1):
    k_optimal = 25

    predicted=0
    usr=int(i.userId)
    mov=int(i.itemId)
    actual=i.rating
        
    all_ratings_for_the_active_item=train[train['itemId']==mov]
    list_of_movies_current_user_has_rated=train[train['userId']==usr]
    similarity=[]
    corres_rate=[]
    
    for ind2,u in enumerate(all_ratings_for_the_active_item.itertuples(),1): 
        c1=[]
        c2=[]
        c_user=int(u.userId)
#         c_item=int(u.itemId)
        list_curuser=train[train['userId']==c_user]
#         list_curitem=train[train['itemId']==c_item]
        xt=pd.merge( list_of_movies_current_user_has_rated,list_curuser,how='inner',on=['itemId'])
        c1=xt['rating_x']
        c2=xt['rating_y']
        c1_list=c1.tolist()
        c2_list=c2.tolist()
        sim=0
        if len(c1_list)!=0:
            sim=(1-spatial.distance.cosine(c1_list,c2_list))#stats.pearsonr(c1_list,c2_list)[0]
        
        similarity.append(sim)
        corres_rate.append(u.rating)
        
    corres_rate = [x for _,x in sorted(zip(similarity,corres_rate),reverse=True)]
    similarity.sort(reverse=True)
#     print (mov)
#     print (similarity)
#     print (corres_rate)
    cs=0
    numc=0
    
    if(len(similarity)<k_optimal):
        k_optimal = len(similarity) 
        
    for k in range(k_optimal+1):        
#         print (k)
        cs+=similarity[k-1]
        numc+=(similarity[k-1]*corres_rate[k-1])
       
    if cs!=0:
        predicted=numc/cs
    else:
        predicted = 0
    
    if mov in target_item_error_in_actual_data:
        target_item_error_in_actual_data[mov][0] = target_item_error_in_actual_data[mov][0] + abs(predicted - calculated_Rating[mov])
        target_item_error_in_actual_data[mov][1] = target_item_error_in_actual_data[mov][1] +1
t2=time.time()        

In [172]:
index

12

In [173]:
print (t2-t1)

0.17502284049987793


In [174]:
target_item_error_in_actual_data

{12: [0.40715590707215776, 2],
 207: [4.9610315534427043, 4],
 216: [0.98157355698276261, 2],
 235: [0.86604060499504065, 2],
 241: [0.21597684835042408, 2]}

### MAE error

In [175]:
for i in target_item_error_in_actual_data:
    print (i,'    :',target_item_error_in_actual_data[i][0]/target_item_error_in_actual_data[i][1])

216     : 0.490786778491
241     : 0.107988424175
235     : 0.433020302498
12     : 0.203577953536
207     : 1.24025788836


## Push Attack Strength Check

### Injecting Attack Profiles

In [176]:
push_rating = 4

In [177]:
attack_ids = list(range(1600,1700))

for attacker in attack_ids:
    for item in window_items:
        a = calculated_Rating[item]
        attack_rating = int(a) if (a-int(a)<0.5) else int(a)+1
        train = train.append(pd.Series([attacker,item,attack_rating], index=['userId', 'itemId' ,'rating']), ignore_index=True)

for attacker in attack_ids:
    for item in target_items:
        train = train.append(pd.Series([attacker,item,push_rating], index=['userId', 'itemId' ,'rating']), ignore_index=True)

In [178]:
train.shape[0]

35042

In [179]:
train.tail(5)

,userId,itemId,rating
35037,1699.0,12.0,4.0
35038,1699.0,207.0,4.0
35039,1699.0,216.0,4.0
35040,1699.0,235.0,4.0
35041,1699.0,241.0,4.0


## Finding error for target items in attack profile injected data

In [180]:
target_item_error_in_actual_data = dict((t_i,[0,0]) for t_i in target_items)
print (target_item_error_in_actual_data)

{216: [0, 0], 241: [0, 0], 235: [0, 0], 12: [0, 0], 207: [0, 0]}


In [181]:
t1=time.time()
for index,i in enumerate(test.itertuples(),1):
    k_optimal = 25

    predicted=0
    usr=int(i.userId)
    mov=int(i.itemId)
    actual=i.rating
        
    all_ratings_for_the_active_item=train[train['itemId']==mov]
    list_of_movies_current_user_has_rated=train[train['userId']==usr]
    similarity=[]
    corres_rate=[]
    
    for ind2,u in enumerate(all_ratings_for_the_active_item.itertuples(),1): 
        c1=[]
        c2=[]
        c_user=int(u.userId)
#         c_item=int(u.itemId)
        list_curuser=train[train['userId']==c_user]
#         list_curitem=train[train['itemId']==c_item]
        xt=pd.merge( list_of_movies_current_user_has_rated,list_curuser,how='inner',on=['itemId'])
        c1=xt['rating_x']
        c2=xt['rating_y']
        c1_list=c1.tolist()
        c2_list=c2.tolist()
        sim=0
        if len(c1_list)!=0:
            sim = (1-spatial.distance.cosine(c1_list,c2_list))#stats.pearsonr(c1_list,c2_list)[0]
        
        similarity.append(sim)
        corres_rate.append(u.rating)
        
    corres_rate = [x for _,x in sorted(zip(similarity,corres_rate),reverse=True)]
    similarity.sort(reverse=True)
#     print (mov)
#     print (similarity)
#     print (corres_rate)
    cs=0
    numc=0
    
    if(len(similarity)<k_optimal):
        k_optimal = len(similarity) 
        
    for k in range(k_optimal+1):        
#         print (k)
        cs+=similarity[k-1]
        numc+=(similarity[k-1]*corres_rate[k-1])
       
    if cs!=0:
        predicted=numc/cs
    else:
        predicted = 0
    
    if mov in target_item_error_in_actual_data:
        target_item_error_in_actual_data[mov][0] = target_item_error_in_actual_data[mov][0] + abs(predicted - calculated_Rating[mov])
        target_item_error_in_actual_data[mov][1] = target_item_error_in_actual_data[mov][1] +1
t2=time.time()        

In [182]:
print (t2-t1)

3.7508482933044434


In [183]:
for i in target_item_error_in_actual_data:
    print (i,'    :',target_item_error_in_actual_data[i][0]/target_item_error_in_actual_data[i][1])

216     : 1.06941824426
241     : 1.04478969849
235     : 1.23046173733
12     : 1.1191073399
207     : 1.06974288847


#### with simple train-test(previously used split) 
1     : 0.256972766762  
10     : 0.250232167679  
12     : 0.158985216812  
207     : 0.193302652524  

1     : 0.277487675027  
10     : 0.30845519241  
12     : 0.254815105493  
207     : 0.212108847568  